# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

In [ ]:
import nrrd
import os

# Path to your NRRD file
data_folder = '/medias/atlas/mouse/CCFv3a/barrel_split'

path = os.path.join(data_folder, 'annotation_ccfv2_l23split_barrelsplit.nrrd')

# Reading the NRRD file
data, header = nrrd.read(path)

# `data` contains the image data as a numpy array
# `header` contains the metadata as a dictionary

print("Data Shape:", data.shape)
print("Header Information:", header)


In [ ]:
import json

f = open(os.path.join(data_folder, 'hierarchy_ccfv2_l23split_barrelsplit.json'))
obj = json.load(f)

region_colors=dict()
def node_color(node):
    node_id = node['id']
    color = node['color_hex_triplet']
    region_colors[node_id] = color
    for child in node['children']:
        node_color(child)

for node in obj['msg']:
    node_color(node)

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

In [ ]:
region_mapping = dict()
region_mapping_index = dict()
region_mapping[0] = 0
i = 0
for key in region_colors.keys():
    region_mapping[key] = i
    region_mapping_index[i] = key
    i += 1

In [ ]:
volume = list()
for z in range(data.shape[2]):
    for y in range(data.shape[1]):
        for x in range(data.shape[0]):
            voxel_value = data[x][y][z]
            volume.append(region_mapping[voxel_value])

In [ ]:
import numpy as np

array = np.array(volume, dtype=np.uint16)

# Write the binary data to a file
volume_filename = os.path.join(data_folder, 'annotation_ccfv2_l23split_barrelsplit.raw')
array.tofile(volume_filename)

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]

cerebellum_granular_layer_ids = [10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732]
cerebellum_molecular_layer_ids =  [10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734]

olfactory_bulb_ids = [507, 212, 228, 236, 244, 220]

medula_ids = [
      773,       781,      1039,       789,      1048,        45,      1069,
      560,       307,        53,       568,        61,       576,       69,
      839,      1098,        76,        77,       83,      1107,       852,
      859,       607,        96,       354, 589508451,       101,       106,
      112,       370,       372,       887,       379,       640,      386,
      642,       903,       135,       136,       651,       395,       653,
      143,       659,       661,       666,       154,       161,       674,
      169,       682,       938,       939,       429,       177,       691,
      437,       185,       955,       445,       701,       193,       963,
      711,       970,       203,       202,       206,       207,       720,
      209,       978,       217,       222,       225,       995,       230,
      235,       765
]

fibers_ids = [960, 1000, 1009, 396, 109]

all_regions = list(set(cerebellum_ids + cerebellum_granular_layer_ids + cerebellum_molecular_layer_ids + olfactory_bulb_ids + medula_ids))


In [ ]:
from bioexplorer import BioExplorer, TransferFunction

colormap_filename = os.path.join(data_folder, 'annotation_ccfv2_l23split_barrelsplit.1dt')
with open(colormap_filename, 'w') as file:
    file.write('%d\n' % len(region_mapping_index))
    for index in region_mapping_index:
        alpha = 0.0
        color = [0,0,0]
        region_id = region_mapping_index[index]
        # if region_id in all_regions and region_id not in fibers_ids:
        if region_id not in[0, 997]:
            color = hex_to_rgb(region_colors[region_id])
            #alpha = 0.05
            alpha = 0.1
        file.write('%f %f %f %f\n' % (color[0] / 255.0, color[1] / 255.0, color[2] / 255.0, alpha))

be = BioExplorer()
model_ids = be.get_model_ids()['ids']
tf = TransferFunction(
    bioexplorer=be, model_id=model_ids[0],
    filename=colormap_filename,
    show_widget=False,
    value_range=[0, len(region_mapping)]
)

In [ ]:
core = be.core_api()
core.set_volume_parameters(
    pre_integration=True, specular=(0,0,0),gradient_shading=True, single_shade=False, gradient_offset=0.5,
    adaptive_sampling=False, adaptive_max_sampling_rate=1.0, sampling_rate=5.0)